In [45]:
from docx import Document
import os
import glob
import pandas as pd
import spacy
from spacy.pipeline import EntityRuler
from collections import defaultdict


In [46]:
# Função para extrair texto de um arquivo .docx
def extract_text(docx_file):
    document = Document(docx_file)
    full_text = []
    for para in document.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

In [47]:
diretorio = '../peticoes'

# Navegar até o diretório
os.chdir(diretorio)

# Coleta dos arquivos pré-corrigidos
arquivos_pre = sorted(glob.glob('preCorrecao*.docx'))

# Verificar se há pelo menos um arquivo
if not arquivos_pre:
    raise FileNotFoundError("Nenhum arquivo pré-corrigido encontrado no diretório especificado.")

# Extrair texto da primeira petição pré-corrigida
texto_pre = extract_text(arquivos_pre[0])

In [48]:
# Carregar o modelo de português do spaCy
nlp = spacy.load('pt_core_news_sm')
ruler = nlp.add_pipe("entity_ruler", before='ner')

# Definir padrões personalizados para ORG e PER
patterns = [
    # 1. Organizações em Caixa Alta
    {
        "label": "ORG",
        "pattern": [
            {"IS_UPPER": True, "OP": "+"},  # Uma ou mais palavras em caixa alta
            {"TEXT": "S.A."}                 # Opcionalmente terminando com "S.A."
        ]
    },
    # 2. Pessoas com Prefixos "Dr." ou "Dra."
    {
        "label": "PER",
        "pattern": [
            {"LOWER": {"IN": ["dr.", "dra."]}},  # Prefixo "Dr." ou "Dra." (case-insensitive)
            {"IS_TITLE": True},                  # Primeira letra maiúscula
            {"IS_TITLE": True, "OP": "+"}        # Nome próprio (uma ou mais palavras com inicial maiúscula)
        ]
    }
]

In [49]:
ruler.add_patterns(patterns)

In [50]:
def anonimizar_texto(texto):
    doc = nlp(texto)
    texto_anonimizado = texto
    for ent in doc.ents:
        if ent.label_ in ['ORG', 'PER']:
            placeholder = f'[{ent.label_}]'
            texto_anonimizado = texto_anonimizado.replace(ent.text, placeholder)
    return texto_anonimizado

In [51]:
# Processar o texto e extrair entidades
texto_pre_anonimizado = anonimizar_texto(texto_pre)
linhas = texto_pre.split('\n')

# Inicializar uma lista para armazenar os dados da tabela
dados_tabela = []


In [52]:
# Iterar sobre cada linha e identificar entidades
for idx, linha in enumerate(linhas):
    if not linha.strip():
        continue  # Pular linhas vazias
    doc_linha = nlp(linha)
    entidades = [(ent.text, ent.label_) for ent in doc_linha.ents]
    
    if entidades:
        # Contar quantas vezes cada label aparece para duplicar colunas se necessário
        label_counts = defaultdict(int)
        for ent_text, ent_label in entidades:
            label_counts[ent_label] += 1
        
        # Criar um dicionário para a linha com as entidades
        linha_dict = {}
        for ent_text, ent_label in entidades:
            count = label_counts[ent_label]
            if count > 1:
                label_counts[ent_label] -= 1
                label_name = f"{ent_label}_{count}"
            else:
                label_name = ent_label
            linha_dict[label_name] = ent_text
        dados_tabela.append(linha_dict)
    else:
        # Se não houver entidades, atribuir a coluna 'TEXT'
        dados_tabela.append({'TEXT': linha})


In [53]:
# Determinar todas as colunas necessárias
todas_colunas = set()
for linha in dados_tabela:
    todas_colunas.update(linha.keys())

# Ordenar as colunas para melhor organização
todas_colunas = sorted(todas_colunas)
# Criar o DataFrame
df_entidades = pd.DataFrame(dados_tabela, columns=todas_colunas)

In [54]:
# Exibir o DataFrame resultante
print(df_entidades)

               LOC        LOC_2 LOC_3    LOC_4  LOC_5 LOC_6  \
0        São Paulo          NaN   NaN      NaN    NaN   NaN   
1              NaN          NaN   NaN      NaN    NaN   NaN   
2             Ilmo          NaN   NaN      NaN    NaN   NaN   
3              NaN          NaN   NaN      NaN    NaN   NaN   
4               SP          NaN   NaN      NaN    NaN   NaN   
5              NaN          NaN   NaN      NaN    NaN   NaN   
6   Prezado Senhor          NaN   NaN      NaN    NaN   NaN   
7              NaN          NaN   NaN      NaN    NaN   NaN   
8      Notificante          NaN   NaN      NaN    NaN   NaN   
9           BR-101   Ubatumirim    SP  Ubatuba  União   STF   
10             NaN          NaN   NaN      NaN    NaN   NaN   
11          BR-101          NaN   NaN      NaN    NaN   NaN   
12           seara  Notificante   NaN      NaN    NaN   NaN   
13      Notificado  Notificante   NaN      NaN    NaN   NaN   
14             NaN          NaN   NaN      NaN    NaN  

In [55]:
import openpyxl

df_entidades.to_excel('tabela__peticao-teste.xlsx', index=False)
